# 📙 LangChain Expression Language (LCEL)

---

## 📌 RunnablePassthrough

### 🎫 RunnablePassthrough 란?

- 입력을 아무 변경 없이 그대로 넘기는 블록 (데이터 전달 역할)

- 용도 : 자리 맡기용, 연결용, 디버깅용, 공백 처리용

- 일반적으로 `RunnableParallel` 과 함께 활용됩니다.

---

## 📌 RunnableParallel

### 🔃 RunnableParallel 란?

- 여러 개의 Runnable을 동시에 실행해주는 블록

- 한 개의 입력을 여러 개로 받고, 각각의 대한 결과물을 여러 개의 결과로 반환

---

### ✅ 검색기 사용에서 RunnablePassthrough를 사용하는 사례 살펴보기

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 문서
docs = [
    "문기는 랭체인이 좋습니다.",
    "민기는 랭체인이 싫습니다.",
    "문기의 직업은 엔지니어입니다.",
    "민기의 직업은 gang입니다."
]
vectorstore = FAISS.from_texts(docs, embedding=OpenAIEmbeddings())

# 벡터 저장소를 검색기로 사용
retriever = vectorstore.as_retriever()

# 템플릿 정의
template = """
Answer the question based only on the following context: {context}
Question: {question}
"""

# 템플릿으로부터 채팅 프롬프트를 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 모델 초기화
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문서를 포맷팅하는 함수
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

# 검색 체인 구성
retrieval_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)

### 실행 및 결과 확인해보기

In [8]:
retrieval_chain.invoke("문기의 직업은 무엇입니까?")

'문기의 직업은 엔지니어입니다.'

In [9]:
retrieval_chain.invoke("민기는 뭐하는 놈이야?")

'민기는 gang입니다.'

In [14]:
retrieval_chain.invoke("RunnablePassthrough은 무엇인가요?")

'주어진 맥락에는 "RunnablePassthrough"에 대한 정보가 포함되어 있지 않습니다. 따라서 답변할 수 없습니다.'

### 정리하기

☑️ "context" : retriever + format_docs 실행 결과

☑️ "question" : 입력값 그대로 들어가야 함

☑️ RunnablePassthrough() : 질문을 아무 수정 없이 그대로 다음 단계에 전달 목적용

☑️ RunnablePassthrough는 필수 인가?\
-> "question" 키를 처리하기 위해 필수 사항임❕\
-> 해당 클래스일 필요는 없지만 이를 대신할 무언가는 필수로 존재해야만 한다.

---

### ✅ 같은 질문에 서로 다른 프롬프트를 병렬로 적용해서 RunnableParallel 알아보기

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
model = ChatOpenAI()

# 프롬프트1 체인 정의
capital_chain = (
    ChatPromptTemplate.from_template("{country} 의 수도는 어디입니까?")
    | model
    | StrOutputParser()
)

# 프롬프트2 체인 정의
area_chain = (
    ChatPromptTemplate.from_template("{country} 의 면적은 얼마입니까?")
    | model
    | StrOutputParser()
)

# 병렬 실행 용 RunnableParallel 객체 생성
map_chain = RunnableParallel(capital=capital_chain, area=area_chain)

### 실행하기

In [6]:
# 질문하기
map_chain.invoke({"country": "대한민국"})

{'capital': '서울입니다.', 'area': '대한민국의 면적은 약 100,210 제곱 킬로미터입니다.'}

---

### ✅ RunnablePassthrough와 RunnableParallel 함께 알아보기

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# 문서
docs = [
    "대한민국의 기후는 사계절이 뚜렷하다.",
    "대한민국은 겨울에 눈이 오고 여름엔 장마가 있다.",
    "대한민국의 수도는 서울이다.",
]
vectorstore = FAISS.from_texts(docs, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

# LLM & 파서
model = ChatOpenAI(model_name="gpt-4o-mini")
parser = StrOutputParser()

# format_docs 함수
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

retriever_chain = retriever | format_docs

# 그대로 질문하는 체인 (Passthrough)
chain_direct = (
    {"question": RunnablePassthrough()}
    | ChatPromptTemplate.from_template("질문: {question}\n정확히 대답해줘.")
    | model
    | parser
)

# RAG 체인 (문서 기반)
chain_rag = (
    {"context": retriever_chain, "question": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(
        "다음 문서를 참고하여 질문에 답하세요.\n문서: {context}\n질문: {question}"
    )
    | model
    | parser
)

# 병렬로 실행 (RunnableParallel)
parallel_chain = RunnableParallel({
    "LLM 단독 답변": chain_direct,
    "LLM & RAG 답변": chain_rag
})

### 실행하기

In [13]:
result = parallel_chain.invoke("대한민국의 기후가 궁금해")

print("\n[LLM 단독 답변]\n")
print(result["LLM 단독 답변"])

print("\n[LLM & RAG 답변]\n")
print(result["LLM & RAG 답변"])


[LLM 단독 답변]

대한민국의 기후는 대체로 온대몬순 기후로 분류됩니다. 사계절이 뚜렷하게 나타나는 것이 특징이며, 각 계절의 기온과 강수량에 따라 다양한 날씨 패턴을 보입니다.

1. **봄(3월~5월)**: 맑고 따뜻한 날씨가 많으며, 꽃이 만개하는 시기입니다. 평균 기온은 약 10도에서 20도 사이입니다. 그러나 봄철에는 황사와 미세먼지가 발생할 수 있습니다.

2. **여름(6월~8월)**: 덥고 습한 날씨가 특징이며, 평균 기온은 25도에서 35도 사이입니다. 이 시기에는 장마가 시작되어 많은 비가 내리며, 습도가 높아져 불쾌지수가 올라갑니다.

3. **가을(9월~11월)**: 맑고 선선한 날씨로, 단풍이 아름답게 물드는 시기입니다. 평균 기온은 약 10도에서 20도 사이로, 쾌적한 날씨가 지속됩니다.

4. **겨울(12월~2월)**: 춥고 건조한 날씨가 이어지며, 평균 기온은 0도에서 -10도 사이입니다. 특히, 북부지역은 눈이 많이 오고, 남부지역은 상대적으로 온난한 겨울을 경험합니다.

전반적으로 대한민국은 사계절의 변화가 뚜렷하여 각 계절마다 다양한 자연 경관과 기후 특성을 즐길 수 있습니다. 또한, 지역에 따라 기후 차이가 있어, 서울과 부산, 제주도 등에서의 기온과 강수량은 다르게 나타날 수 있습니다.

[LLM & RAG 답변]

대한민국의 기후는 사계절이 뚜렷합니다. 겨울에는 눈이 내리고, 여름에는 장마가 있습니다. 이러한 특징 덕분에 각 계절마다 다양한 날씨와 풍경을 경험할 수 있습니다.


---
---

## 📌 Runnable 구조(그래프) 검토

### 📈 Runnable 구조(그래프) 검토
- Runnable의 흐름을 이해하기 위한 그래프 그려보기

### 라이브러리 설치

In [ ]:
# 필요 라이브러리 설치
!pip install -qU faiss-cpu tiktoken

# 그래프 라이브러리 설치
!pip install -qU grandalf

### 일반적인 체인 구성 샘플

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    # 텍스트 데이터로부터 FAISS 벡터 저장소를 생성합니다.
    ["Teddy는 AI 엔지니어입니다.", "Teddy는 프로그래밍을 좋아합니다!"],
    embedding=OpenAIEmbeddings(),
)

# 벡터 저장소를 기반으로 retriever를 생성합니다.
retriever = vectorstore.as_retriever()

template = """다음 맥락만을 토대로 질문에 답하세요.:
{context}  

Question: {question}"""

# 템플릿을 기반으로 ChatPromptTemplate을 생성합니다.
prompt = ChatPromptTemplate.from_template(
    template
)

# ChatOpenAI 모델을 초기화합니다.
model = ChatOpenAI(model="gpt-4o-mini")

# chain 을 생성합니다.
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

### Runnable의 그래프 얻기

In [20]:
# 체인의 그래프에서 노드 가져오기
chain.get_graph().nodes

{'9e99e544dd5f46d39b9827e4f250790b': Node(id='9e99e544dd5f46d39b9827e4f250790b', name='Parallel<context,question>Input', data=<class 'langchain_core.runnables.base.RunnableParallel<context,question>Input'>, metadata=None),
 '1961b2b62b8d496f86fb81ffb2d0029e': Node(id='1961b2b62b8d496f86fb81ffb2d0029e', name='Parallel<context,question>Output', data=<class 'langchain_core.utils.pydantic.RunnableParallel<context,question>Output'>, metadata=None),
 '2b3e089446224bdda73e584caed61770': Node(id='2b3e089446224bdda73e584caed61770', name='VectorStoreRetriever', data=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1694c8690>, search_kwargs={}), metadata=None),
 'dd95ff52bf2947749987f4c8ad52350e': Node(id='dd95ff52bf2947749987f4c8ad52350e', name='Passthrough', data=RunnablePassthrough(), metadata=None),
 '9f1ac3dec1f74dcd9a7a92d1dd28ffe5': Node(id='9f1ac3dec1f74dcd9a7a92d1dd28ffe5', name='ChatPromptTemplate', data=ChatP

- 노드(Node): 각각의 처리 단계 (예: retriever, prompt, model 등)

In [ ]:
# 체인의 그래프에서 엣지를 가져오기
chain.get_graph().edges

[Edge(source='c525746f8cf2494eb90f2c08097eded9', target='5e45775b2cb444c0a674d3d5004a1760', data=None, conditional=False),
 Edge(source='5e45775b2cb444c0a674d3d5004a1760', target='863efcd269944b3486ace0c82316e1f1', data=None, conditional=False),
 Edge(source='c525746f8cf2494eb90f2c08097eded9', target='d50f4826cfc74a28bdc8e3aaa3e31292', data=None, conditional=False),
 Edge(source='d50f4826cfc74a28bdc8e3aaa3e31292', target='863efcd269944b3486ace0c82316e1f1', data=None, conditional=False),
 Edge(source='863efcd269944b3486ace0c82316e1f1', target='46cae50d6a4e4934a1440d4afc4be280', data=None, conditional=False),
 Edge(source='46cae50d6a4e4934a1440d4afc4be280', target='f667874687d14d7fbe43aa9ffe4812da', data=None, conditional=False),
 Edge(source='ed2d1a69a5634b40b8f5e711d7004b53', target='6270c2309d4b4d4390a2cbc297860425', data=None, conditional=False),
 Edge(source='f667874687d14d7fbe43aa9ffe4812da', target='ed2d1a69a5634b40b8f5e711d7004b53', data=None, conditional=False)]

- 엣지(Edge): 그 처리 단계들 사이를 연결하는 “화살표”나 “선”

### 그래프 출력하기

In [ ]:
# 체인의 그래프를 ASCII 형식으로 출력
chain.get_graph().print_ascii()

           +---------------------------------+         
           | Parallel<context,question>Input |         
           +---------------------------------+         
                    **               **                
                 ***                   ***             
               **                         **           
+----------------------+              +-------------+  
| VectorStoreRetriever |              | Passthrough |  
+----------------------+              +-------------+  
                    **               **                
                      ***         ***                  
                         **     **                     
           +----------------------------------+        
           | Parallel<context,question>Output |        
           +----------------------------------+        
                             *                         
                             *                         
                             *                  

### ☑️ 정리하기

1.	ParallelInput: 체인의 맨 앞에서 "context"와 "question" 두 개의 key에 대해 동시에 처리 준비

2.	VectorStoreRetriever: "context" 값 생성을 위해 retriever 실행 (벡터 검색)

3.	Passthrough: "question"은 입력값 그대로 통과

4.	ParallelOutput: "context" + "question" → 딕셔너리로 묶어서 다음 단계로 넘김

5.	ChatPromptTemplate: {context}\nQuestion: {question} 형태의 프롬프트 생성

6.	ChatOpenAI: 프롬프트를 LLM(GPT-4o-mini)에 전달 → 응답 생성

7.	StrOutputParser: 모델 응답을 파싱해서 문자열로 변환

8.	StrOutputParserOutput: 최종 출력
